In [ ]:
!pip install rich==6.2.0
!pip install cobra
import cobra.test
from cobra import Model, Reaction, Metabolite
from cobra.util.solver import linear_reaction_coefficients

     |████████████████████████████████| 153kB 14.7MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 2.4MB 18.5MB/s 
     |████████████████████████████████| 10.1MB 42.6MB/s 
     |████████████████████████████████| 2.1MB 48.1MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 6.2MB 28.7MB/s 
     |████████████████████████████████| 133kB 55.5MB/s 
     |████████████████████████████████| 71kB 2.1MB/s 
     |████████████████████████████████| 102kB 11.3MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 552kB 54.9MB/s 
     |████████████████████████████████| 61kB 8.5MB/s 


In [ ]:
import cobra.test
import os
from os.path import join

data_dir = cobra.test.data_dir

print("mini test files: ")
print(", ".join(i for i in os.listdir(data_dir)
                if i.startswith("mini")))


mini test files: 
mini.mat, mini_cobra.xml, mini_fbc2.xml.gz, mini_fbc2.xml, mini.pickle, mini_fbc1.xml, mini.yml, mini.json, mini_fbc2.xml.bz2


Load the Synechocystis core model:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model = cobra.io.read_sbml_model(join(data_dir, "/content/drive/MyDrive/Colab Notebooks/MODEL1507180046_url.xml"))
model_base = cobra.io.read_sbml_model(join(data_dir, "/content/drive/MyDrive/Colab Notebooks/MODEL1507180046_url.xml"))


model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0002849,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0002054,0,0.00%
cu2_e,EX_cu2_e,0.00019,0,0.00%
fe2_e,EX_fe2_e,0.0004727,0,0.00%
fe3_e,EX_fe3_e,0.0004304,0,0.00%
glc_DASH_D_e,EX_glc_DASH_D_e,0.85,6,100.00%
h_e,EX_h_e,0.6286,0,0.00%
k_e,EX_k_e,0.01068,0,0.00%
mg2_e,EX_mg2_e,0.001816,0,0.00%
mn2_e,EX_mn2_e,0.0001913,0,0.00%


Checking if autotrophic conditions are right, with FBA and pFBA:

In [ ]:
model.objective = "Ec_biomass_SynAuto"
model.reactions.get_by_id("EX_photon_e").lower_bound = -100
model.reactions.get_by_id("EX_glc_DASH_D_e").lower_bound = 0
model.reactions.get_by_id("EX_hco3_e").lower_bound = -3.7
sol = model.optimize()
model.summary()

Objective value at 0.884 (growth rate) for autotrophic conditions. Comparable to Nogales paper. 

In [ ]:
pFBA = cobra.flux_analysis.pfba(model)
abs(sol.fluxes["Ec_biomass_SynAuto"] - pFBA.fluxes["Ec_biomass_SynAuto"])

pFBA performed to confirm that autotrophic conditions are correct. 

## Comparison with other research:

Autotrophic conditions and demand function for G3P:

In [ ]:
model_base.objective = "Ec_biomass_SynAuto"
model_base.reactions.get_by_id("EX_photon_e").lower_bound = -100 
model_base.reactions.get_by_id("EX_glc_DASH_D_e").lower_bound = 0
model_base.reactions.get_by_id("EX_hco3_e").lower_bound = -3.7

model_base.add_boundary(model_base.metabolites.get_by_id("g3p_c"), type="demand")
model_base.objective = "DM_g3p_c"
sol_base = model_base.optimize()
model_base.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
h_e,EX_h_e,3.7,0,0.00%
hco3_e,EX_hco3_e,3.7,1,100.00%
photon_e,EX_photon_e,32.89,0,0.00%
pi_e,EX_pi_e,1.233,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
g3p_c,DM_g3p_c,-1.233,3,100.00%
h2o_e,EX_h2o_e,-1.233,0,0.00%
o2_e,EX_o2_e,-3.7,0,0.00%


In [ ]:
model_ndh = model_base

Simplified photosynthetic model:

In [ ]:
# Autotrophic conditions and G3P demand reaction
model.objective = "Ec_biomass_SynAuto"
model.reactions.get_by_id("EX_photon_e").lower_bound = -100 
model.reactions.get_by_id("EX_glc_DASH_D_e").lower_bound = 0
model.reactions.get_by_id("EX_hco3_e").lower_bound = -3.7
model.add_boundary(model.metabolites.get_by_id("g3p_c"), type="demand")

# Simplified photosynthetic model 
# Contraining cytochrome c6-dependent reactions
model.reactions.get_by_id("CBFC2").lower_bound = 0
model.reactions.get_by_id("CBFC2").upper_bound = 0
model.reactions.get_by_id("PSI_2").lower_bound = 0
model.reactions.get_by_id("PSI_2").upper_bound = 0
model.reactions.get_by_id("CYO1b_syn").lower_bound = 0
model.reactions.get_by_id("CYO1b_syn").upper_bound = 0

# Contraining NADH-dependent reactions
model.reactions.get_by_id("NDH1_2p").lower_bound = 0
model.reactions.get_by_id("NDH1_2p").upper_bound = 0
model.reactions.get_by_id("NDH1_2u").lower_bound = 0
model.reactions.get_by_id("NDH1_2u").upper_bound = 0

# Constraining peryplasmatic reactions
model.reactions.get_by_id("NDH1_1p").lower_bound = 0
model.reactions.get_by_id("NDH1_1p").upper_bound = 0
model.reactions.get_by_id("CYO1b2pp_syn").lower_bound = 0
model.reactions.get_by_id("CYO1b2pp_syn").upper_bound = 0
model.reactions.get_by_id("CBFCpp").lower_bound = 0
model.reactions.get_by_id("CBFCpp").upper_bound = 0
model.reactions.get_by_id("CYTBDpp").lower_bound = 0
model.reactions.get_by_id("CYTBDpp").upper_bound = 0

# Assumed irreversibility of ferredoxin NADPH reductase under autotrophic conditions
model.reactions.get_by_id("FNOR").lower_bound = 0

# Assumed no excretion of CO2 under autotrophic conditions
model.reactions.get_by_id("EX_co2_e").upper_bound = 0

# Building model lacking AEF pathways
model.reactions.get_by_id("NDH1_1u").lower_bound = 0
model.reactions.get_by_id("NDH1_1u").upper_bound = 0
model.reactions.get_by_id("NDH2_syn").lower_bound = 0
model.reactions.get_by_id("NDH2_syn").upper_bound = 0
model.reactions.get_by_id("NDH1_3u").lower_bound = 0
model.reactions.get_by_id("NDH1_3u").upper_bound = 0
model.reactions.get_by_id("FQR").lower_bound = 0
model.reactions.get_by_id("FQR").upper_bound = 0
model.reactions.get_by_id("CYO1b2_syn").lower_bound = 0
model.reactions.get_by_id("CYO1b2_syn").upper_bound = 0
model.reactions.get_by_id("CYTBDu").lower_bound = 0
model.reactions.get_by_id("CYTBDu").upper_bound = 0
model.reactions.get_by_id("H2ASE_syn").lower_bound = 0
model.reactions.get_by_id("H2ASE_syn").upper_bound = 0
model.reactions.get_by_id("MEHLER").lower_bound = 0
model.reactions.get_by_id("MEHLER").upper_bound = 0

# 

In [ ]:
model.objective = "DM_g3p_c"
model.optimize()
print(model.summary())

Objective
1.0 DM_g3p_c = 1.233333333333356

Uptake
------
Metabolite     Reaction  Flux  C-Number   C-Flux
       h_e       EX_h_e   3.7         0    0.00%
    hco3_e    EX_hco3_e   3.7         1  100.00%
  photon_e  EX_photon_e 33.11         0    0.00%
      pi_e      EX_pi_e 1.233         0    0.00%

Secretion
---------
Metabolite  Reaction   Flux  C-Number   C-Flux
     g3p_c  DM_g3p_c -1.233         3  100.00%
     h2o_e  EX_h2o_e -1.233         0    0.00%
      o2_e   EX_o2_e   -3.7         0    0.00%



Mutant M1 knockout genes contraint for simple model.

In [ ]:
model_ndh.reactions.get_by_id("NDH1_2p").lower_bound = 0
model_ndh.reactions.get_by_id("NDH1_2p").upper_bound = 0
model_ndh.reactions.get_by_id("NDH1_2u").lower_bound = 0
model_ndh.reactions.get_by_id("NDH1_2u").upper_bound = 0
model_ndh.reactions.get_by_id("NDH2_syn").lower_bound = 0
model_ndh.reactions.get_by_id("NDH2_syn").upper_bound = 0
model_ndh.reactions.get_by_id("GLYCTO1").lower_bound = 0
model_ndh.reactions.get_by_id("GLYCTO1").upper_bound = 0
model_ndh.reactions.get_by_id("GLUSx").lower_bound = 0
model_ndh.reactions.get_by_id("GLUSx").upper_bound = 0
model_ndh.reactions.get_by_id("MDH").lower_bound = 0
model_ndh.reactions.get_by_id("MDH").upper_bound = 0
model_ndh.reactions.get_by_id("POR_syn").lower_bound = 0
model_ndh.reactions.get_by_id("POR_syn").upper_bound = 0
model_ndh.reactions.get_by_id("NADTRHD").lower_bound = 0
model_ndh.reactions.get_by_id("NADTRHD").upper_bound = 0
model_ndh.objective = "DM_g3p_c"
model_ndh.optimize()
model_ndh.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
h_e,EX_h_e,3.7,0,0.00%
hco3_e,EX_hco3_e,3.7,1,100.00%
photon_e,EX_photon_e,32.89,0,0.00%
pi_e,EX_pi_e,1.233,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
g3p_c,DM_g3p_c,-1.233,3,100.00%
h2o_e,EX_h2o_e,-1.233,0,0.00%
o2_e,EX_o2_e,-3.7,0,0.00%
